# Chap3: word2vec

In [19]:
import os
from pathlib import Path
import sys

sys.path.append(os.path.join(Path().resolve(), 'deep-learning-from-scratch-2'))

## 3.1 推論ベースの手法と NN

- カウントベースだと単語の種類が増えると行列が巨大になってしまい、次元を削減するための特異値分解 (SVD) も $$O(n^3)$$ の計算量
- マスクされた部分の単語を推測する推論ベースの手法
- 推論ベースでは NN に固定長の vector を入力するため、例えば文を one-hot 表現にして入力

## 3.2 word2vec

- word2vec では continuous bag-of-words (CBOW) と skip-gram が使われる (どっちも NN)

### 3.2.1 CBOW の推論処理

- CBOW は context から target (中央の単語) を推測する
- 入力層は context として入力する単語の数
- 入力層から中間層に変換する全結合層の重みが単語の分散表現となる
  - これは学習が進むと更新されていき、「意味」もエンコードされるようになっていく
- CBOW の推論処理のコードは下記


In [20]:
from ch03 import cbow_predict

cbow_predict??

Type:        module
String form: <module 'ch03.cbow_predict' from '/Users/oka/git/zero-deep-2/deep-learning-from-scratch-2/ch03/cbow_predict.py'>
File:        ~/git/zero-deep-2/deep-learning-from-scratch-2/ch03/cbow_predict.py
Source:     
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul


# サンプルのコンテキストデータ
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 重みの初期化
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# レイヤの生成
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 順伝搬
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)
print(s)
